# 1. 张量（Tensor）基础
PyTorch 的核心数据结构是 张量（Tensor），类似于 NumPy 的数组，但支持 GPU 加速。

**创建张量**

In [3]:
import torch

# 创建未初始化的张量
x = torch.empty(2, 3)  # 2行3列的空张量

# 创建随机张量（范围 [0, 1)）
x = torch.rand(2, 3)  

# 创建全零张量
x = torch.zeros(2, 3, dtype=torch.long)  # 指定数据类型为长整型

# 从 Python 列表创建
x = torch.tensor([1, 2, 3])  

# 从 NumPy 数组创建
import numpy as np
np_array = np.array([4, 5, 6])
x = torch.from_numpy(np_array)

**张量属性**

In [4]:
print(x.shape)   # 形状
print(x.dtype)   # 数据类型
print(x.device)  # 所在设备（CPU/GPU）

torch.Size([3])
torch.int64
cpu


**张量运算**

In [7]:
import torch

# 创建 2x3 的张量 x 和 y
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
y = torch.tensor([[7, 8, 9], [10, 11, 12]])
z = x + y        # 等价于 torch.add(x, y)
print(z)
torch.add(x, y, out=z)  # 指定输出张量
print(z)


# 索引（与 NumPy 类似）
print(x[:, 1])   # 所有行的第1列

# 调整形状
x = x.view(6)    # 变为 6 元素的一维张量
x = x.reshape(2, 3)  # 调整形状为 2x3

# 转换为 NumPy
np_array = x.numpy()  # 若 x 在 CPU 上

tensor([[ 8, 10, 12],
        [14, 16, 18]])
tensor([[ 8, 10, 12],
        [14, 16, 18]])
tensor([2, 5])


## 2. 自动求导（Autograd）

In [7]:
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2 + 3 * x + 1
y.backward()     # 计算梯度
print(x.grad)    # 输出 dy/dx = 2x + 3 = 7.0

tensor(7.)


In [8]:
with torch.no_grad():
    y = x * 2    # 此操作不记录梯度

## 3. 模型构建（nn.Module）

**定义模型**

In [9]:
import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(784, 256)  # 输入层到隐藏层
        self.fc2 = nn.Linear(256, 10)   # 隐藏层到输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNet()

**模型参数与设备转移**

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # 将模型移动到 GPU（如果可用）

SimpleNet(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

## 4.数据处理 

In [18]:
from torch.utils.data import Dataset, DataLoader

# 假设 data 和 labels 是你的数据集和标签
data = [[1, 2], [3, 4], [5, 6]]  # 示例数据
labels = [0, 1, 0]  # 示例标签

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 创建数据集实例
dataset = CustomDataset(data, labels)

# 创建 DataLoader 实例
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 测试 DataLoader
for batch in dataloader:
    print(batch)

[[tensor([3, 5, 1]), tensor([4, 6, 2])], tensor([1, 0, 0])]


## 5.训练

In [20]:
criterion = nn.CrossEntropyLoss()  # 交叉熵损失（分类任务）
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam 优化器

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# 假设 data 和 labels 是你的数据集和标签
data = torch.tensor([[1.0], [2.0], [3.0], [4.0]], dtype=torch.float32)  # 示例数据，转换为张量
labels = torch.tensor([[2.0], [4.0], [6.0], [8.0]], dtype=torch.float32)  # 示例标签，转换为张量

# 自定义数据集
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]  # 返回张量

# 创建数据集实例
dataset = CustomDataset(data, labels)

# 创建 DataLoader 实例
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 定义一个非常简单的线性模型
class TinyModel(nn.Module):
    def __init__(self):
        super(TinyModel, self).__init__()
        self.linear = nn.Linear(1, 1)  # 输入维度 1，输出维度 1

    def forward(self, x):
        return self.linear(x)

# 定义设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 初始化模型、损失函数和优化器
model = TinyModel().to(device)  # 将模型移动到设备
criterion = nn.MSELoss()  # 均方误差损失函数
optimizer = optim.SGD(model.parameters(), lr=0.01)  # 随机梯度下降优化器

# 训练循环
for epoch in range(10):
    model.train()  # 设置为训练模式
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)  # 将数据移动到设备

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 反向传播与优化
        optimizer.zero_grad()  # 清空梯度
        loss.backward()        # 计算梯度
        optimizer.step()       # 更新参数

    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")

# 测试模型
model.eval()  # 设置为评估模式
with torch.no_grad():
    test_input = torch.tensor([[5.0]], dtype=torch.float32).to(device)
    predicted_output = model(test_input)
    print(f"Predicted output for input 5.0: {predicted_output.item():.4f}")

Epoch [1/10], Loss: 43.7551
Epoch [2/10], Loss: 5.2969
Epoch [3/10], Loss: 5.1233
Epoch [4/10], Loss: 1.1414
Epoch [5/10], Loss: 1.5878
Epoch [6/10], Loss: 0.5372
Epoch [7/10], Loss: 0.6586
Epoch [8/10], Loss: 0.3186
Epoch [9/10], Loss: 0.0550
Epoch [10/10], Loss: 0.0074
Predicted output for input 5.0: 9.6314
